In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [ ]:
# Load and preprocess the dataset
grid_size = 30
num_samples = 10000
num_selected_samples = 1000
selected_designs_masked, _ = create_dataset(num_samples, num_selected_samples)

# Split the dataset into train and validation sets
train_data, val_data = train_test_split(selected_designs_masked, test_size=0.1, random_state=42)

# Normalize the data
train_data = np.array(train_data) / 3.0  # Normalizing between 0 and 1
val_data = np.array(val_data) / 3.0

# Convert data to PyTorch tensors
train_data = torch.FloatTensor(train_data)
val_data = torch.FloatTensor(val_data)

In [ ]:
# Define VAE architecture
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc_mean = nn.Linear(hidden_size, latent_size)
        self.fc_logvar = nn.Linear(hidden_size, latent_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        mean = self.fc_mean(x)
        logvar = self.fc_logvar(x)
        return mean, logvar

class Decoder(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, latent_size)
        self.decoder = Decoder(latent_size, hidden_size, input_size)

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x):
        mean, logvar = self.encoder(x)
        z = self.reparameterize(mean, logvar)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mean, logvar


In [ ]:
# Instantiate VAE
input_size = grid_size * grid_size
hidden_size = 128
latent_size = 16
vae = VAE(input_size, hidden_size, latent_size)

# Define loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [ ]:
# Training loop
num_epochs = 50
batch_size = 64
learning_rate = 1e-3

optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
    for i in range(0, len(train_data), batch_size):
        batch = train_data[i:i+batch_size]
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(batch.view(-1, input_size))
        loss = loss_function(recon_batch, batch.view(-1, input_size), mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss / len(train_data):.4f}')


In [ ]:
# Generate samples
vae.eval()
with torch.no_grad():
    num_samples_to_generate = 5
    z_samples = torch.randn(num_samples_to_generate, latent_size)
    generated_samples = vae.decoder(z_samples).view(-1, grid_size, grid_size).numpy()
